In [11]:
# automated ref adding for lineage-note.txt, written by Xu Zou

# old code, do not use!
import bs4
from bs4 import BeautifulSoup
from urllib import request
import os
import json
# Use AngieHinrich's better query code for milestones

# Build milestone_list from API query that lists all milestones in a repo.
milestone_list = []
milestone_already=[]
milestone_title_dict={}
listd=os.listdir()
if not("autoref" in listd):
    os.mkdir("autoref")
lista=os.listdir("autoref")
if ("milestone.txt" in lista):
    w=open("autoref/milestone.txt",'r')
    milestones=w.readlines()
    for line in milestones:
        linsp=line.split()
        if len(linsp)>2:
            issue=linsp[0]
            name=linsp[1]
            milestone_list.append([linsp[0],linsp[1],linsp[2]])
            if linsp[1]!='withdrawn':
                milestone_title_dict[linsp[1]]=[linsp[2],linsp[0]]
            milestone_already.append(issue)
    w.close()

w=open("autoref/milestone.txt",'w')
for item in milestone_list:
    print(item[0],item[1],item[2],file=w)
item_count=0
for repo in ['cov-lineages/pango-designation', 'sars-cov-2-variants/lineage-proposals']:
    if repo == 'sars-cov-2-variants/lineage-proposals':
        prefix = 'sars#'
    else:
        prefix = ''
    # Request pages starting with 1, until we get an empty result list.
    page = 1
    while 1:
        url = f'https://api.github.com/repos/{repo}/milestones?per_page=100&page={page}'
        res_text = request.urlopen(url).read().decode('utf-8')
        res_list = json.loads(res_text)
        
        if len(res_list) == 0:
            break
        # Each element of this page's result list includes the milestone number and title.  Parse out lineage names from title.
        for milestone_info in res_list:
            #print(milestone_info)
            milestone = milestone_info['number']
            milestone=prefix+str(milestone)
            if milestone not in milestone_already:
                try:
                    title = milestone_info['title']
                    title_words = title.replace('. ','.').replace(',', ' ').split()
                    title = title.replace(' ', '+')
                    
                    for item in title_words:
                        milestone_list.append([milestone, item, title])
                        if linsp[1]!='withdrawn':
                            milestone_title_dict[item]=[title,milestone]
                        item_count+=1
                        print(milestone, item,title,file=w)
                        if item_count%10==0:
                            print("processed ",item_count,"milestones")
                except Exception as e:
                    print(milestone, e)
        page += 1
w.close()


801 HTTP Error 404: Not Found
10 HTTP Error 404: Not Found
59 HTTP Error 404: Not Found
Reading Milestone in sars-cov-2-variants  60


In [16]:

                
def read_milestone_issues(name):
    # or sars-cov-2 proj if it starts to add milestones
    
    # first, deal with multiple names
    #print(name)
    real_milestone=milestone_title_dict[name]
    di='cov-lineages/pango-designation'
    if 'sars' in real_milestone[1]:
        di='sars-cov-2-variants/lineage-proposals'
    actual_milestone='"'+real_milestone[0]+'"'
    url="https://github.com/"+di+"/issues?q=milestone:"+actual_milestone
    res=request.urlopen(url)
    rp=res.read()
    rp=rp.decode('utf-8')
    html_soup=BeautifulSoup(rp,"html")
    html=html_soup.find_all(class_="Link--primary v-align-middle no-underline h4 js-navigation-open markdown-title" )
    try:
        id_str=html[0]['id']
        outid=id_str.split("_")[1]
        if 'sars' in real_milestone[1]:
            outid='sars#'+outid
        return outid
    except Exception as e:
        print(name,e)
        return '0'



In [17]:
def examine_line(line):
    # deal with issues/ situation
    if 'issues/' in line:
        linsp=line.split('issues/')
        rel=linsp[0]+'issues/'
        pos=0
        while (linsp[1][pos]>='0' and linsp[1][pos]<='9' and pos<len(linsp[1])):
            pos+=1
        rel=rel+linsp[1][:pos]+'\n'
        return rel
    if '#' in line:
        return line
    return ''
    

In [20]:
# read lineage_notes.txt


def milestone2issue(num):
    return 0
w=open("lineage_notes.txt",'r')
lineage_notes=w.readlines()
w.close()
milestone_tobecheck=[]


changed_items=0
missing_milestones=[]
for num in range(1,len(lineage_notes)):
    line=lineage_notes[num]
    linsp=line.split()
    if (len(linsp)>=2):
        title=linsp[0]
        # for all lineages without an issue label but with a milestone
        if not('#' in line or 'issues/' in line):
            if title in milestone_title_dict:
                # this time we need to deal with the milestone 
                
                issue_num=read_milestone_issues(title)
                if issue_num!='0':
                    if 'sars' in issue_num:
                        issue_num=issue_num.replace('sars','sars-cov-2-variants/lineage-proposals')
                    else:
                        issue_num='#'+issue_num
                        
                    missing_milestones.append([title,issue_num])
                    print(missing_milestones[-1])
                
                    
#rewrite lineage_notes.txt
w=open("autoref/missing_refs.txt",'w')
for item in missing_milestones:
    print(item[0],item[1],file=w)
w.close()
        


CM.5.1 list index out of range
['BN.1.3.13', 'sars-cov-2-variants/lineage-proposals#218']
B.1.575 list index out of range
XBB.1.5 list index out of range
['HQ.1', 'sars-cov-2-variants/lineage-proposals#170']
XBB.1.5.99 list index out of range
FL.1.3 list index out of range
['FP.2.1', '#2012']
['FY.4.2', 'sars-cov-2-variants/lineage-proposals#487']


In [21]:
#final rewrite of lineage_notes.txt
w=open("autoref/missing_refs.txt",'r')
data=w.readlines()
w.close()
missing_refs={}
for line in data:
    linsp=line.split()
    missing_refs[linsp[0]]=linsp[1].replace('\n','')

w=open("lineage_notes.txt",'r')
data=w.readlines()
w.close()
normal_count=0
sub_count=0
for i in range(len(data)):
    linsp=data[i].split()
    if linsp[0] in missing_refs:
        data[i]=data[i].replace('\n','')+" From "+missing_refs[linsp[0]]+'\n'
        if 'sars' in missing_refs[linsp[0]]:
            sub_count+=1
        else:
            normal_count+=1

print("Add ",normal_count," references from cov-lineages/pango-designation.")
print("Add ",sub_count," references from sars-cov-2-variants/lineage-proposals.")

w=open("lineage_notes.txt",'w',newline='\n')
w.writelines(data)
w.close()


Add  1  references from cov-lineages/pango-designation.
Add  3  references from sars-cov-2-variants/lineage-proposals.


In [2]:
# fix some bug to adapt to new manual format that does not contain #. 
w=open("lineage_notes.txt",'r')
lines=w.readlines()
for i in range(len(lines)):
    em=examine_line(lines[i])
    if em!='':
        lines[i]=em
w.close()
w=open("lineage_notes.txt",'w',newline='\n')
w.writelines(lines)
w.close()


In [6]:
import json
from urllib import request
...
# Build milestone_list from API query that lists all milestones in a repo.
milestone_list = []
milestone_already=[]
for repo in ['cov-lineages/pango-designation', 'sars-cov-2-variants/lineage-proposals']:
    if repo == 'sars-cov-2-variants/lineage-proposals':
        prefix = 'sars#'
    else:
        prefix = ''
    # Request pages starting with 1, until we get an empty result list.
    page = 1
    while 1:
        url = f'https://api.github.com/repos/{repo}/milestones?per_page=100&page={page}'
        res_text = request.urlopen(url).read().decode('utf-8')
        res_list = json.loads(res_text)
        
        if len(res_list) == 0:
            break
        # Each element of this page's result list includes the milestone number and title.  Parse out lineage names from title.
        for milestone_info in res_list:
            print(milestone_info)
            milestone = milestone_info['number']
            if milestone not in milestone_already:
                try:
                    title = milestone_info['title']
                    title_words = title.replace('. ','.').replace(',', ' ').split()
                    title = title.replace(' ', '+')
                    for item in title_words:
                        milestone=prefix+milestone
                        item = item
                        milestone_list.append([milestone, item, title])
                        print(milestone, item)
                except Exception as e:
                    print(milestone, e)
        page += 1

{'url': 'https://api.github.com/repos/cov-lineages/pango-designation/milestones/1', 'html_url': 'https://github.com/cov-lineages/pango-designation/milestone/1', 'labels_url': 'https://api.github.com/repos/cov-lineages/pango-designation/milestones/1/labels', 'id': 6428650, 'node_id': 'MDk6TWlsZXN0b25lNjQyODY1MA==', 'number': 1, 'title': 'C.1', 'description': None, 'creator': {'login': 'rambaut', 'id': 686405, 'node_id': 'MDQ6VXNlcjY4NjQwNQ==', 'avatar_url': 'https://avatars.githubusercontent.com/u/686405?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/rambaut', 'html_url': 'https://github.com/rambaut', 'followers_url': 'https://api.github.com/users/rambaut/followers', 'following_url': 'https://api.github.com/users/rambaut/following{/other_user}', 'gists_url': 'https://api.github.com/users/rambaut/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/rambaut/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/rambaut/subscriptions', 'organiz

{'url': 'https://api.github.com/repos/cov-lineages/pango-designation/milestones/102', 'html_url': 'https://github.com/cov-lineages/pango-designation/milestone/102', 'labels_url': 'https://api.github.com/repos/cov-lineages/pango-designation/milestones/102/labels', 'id': 7072623, 'node_id': 'MI_kwDOEGMSVc4Aa-tv', 'number': 102, 'title': 'B.1.637', 'description': None, 'creator': {'login': 'AngieHinrichs', 'id': 186983, 'node_id': 'MDQ6VXNlcjE4Njk4Mw==', 'avatar_url': 'https://avatars.githubusercontent.com/u/186983?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/AngieHinrichs', 'html_url': 'https://github.com/AngieHinrichs', 'followers_url': 'https://api.github.com/users/AngieHinrichs/followers', 'following_url': 'https://api.github.com/users/AngieHinrichs/following{/other_user}', 'gists_url': 'https://api.github.com/users/AngieHinrichs/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/AngieHinrichs/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.gith

{'url': 'https://api.github.com/repos/cov-lineages/pango-designation/milestones/202', 'html_url': 'https://github.com/cov-lineages/pango-designation/milestone/202', 'labels_url': 'https://api.github.com/repos/cov-lineages/pango-designation/milestones/202/labels', 'id': 7497695, 'node_id': 'MI_kwDOEGMSVc4Acmff', 'number': 202, 'title': 'AY.39.1.3', 'description': '', 'creator': {'login': 'chrisruis', 'id': 47785131, 'node_id': 'MDQ6VXNlcjQ3Nzg1MTMx', 'avatar_url': 'https://avatars.githubusercontent.com/u/47785131?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/chrisruis', 'html_url': 'https://github.com/chrisruis', 'followers_url': 'https://api.github.com/users/chrisruis/followers', 'following_url': 'https://api.github.com/users/chrisruis/following{/other_user}', 'gists_url': 'https://api.github.com/users/chrisruis/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/chrisruis/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/chrisruis/s

{'url': 'https://api.github.com/repos/cov-lineages/pango-designation/milestones/302', 'html_url': 'https://github.com/cov-lineages/pango-designation/milestone/302', 'labels_url': 'https://api.github.com/repos/cov-lineages/pango-designation/milestones/302/labels', 'id': 8031434, 'node_id': 'MI_kwDOEGMSVc4AeozK', 'number': 302, 'title': 'BA.5.4', 'description': None, 'creator': {'login': 'InfrPopGen', 'id': 50238498, 'node_id': 'MDQ6VXNlcjUwMjM4NDk4', 'avatar_url': 'https://avatars.githubusercontent.com/u/50238498?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/InfrPopGen', 'html_url': 'https://github.com/InfrPopGen', 'followers_url': 'https://api.github.com/users/InfrPopGen/followers', 'following_url': 'https://api.github.com/users/InfrPopGen/following{/other_user}', 'gists_url': 'https://api.github.com/users/InfrPopGen/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/InfrPopGen/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/InfrP

{'url': 'https://api.github.com/repos/cov-lineages/pango-designation/milestones/402', 'html_url': 'https://github.com/cov-lineages/pango-designation/milestone/402', 'labels_url': 'https://api.github.com/repos/cov-lineages/pango-designation/milestones/402/labels', 'id': 8312582, 'node_id': 'MI_kwDOEGMSVc4AftcG', 'number': 402, 'title': 'BK.1', 'description': None, 'creator': {'login': 'InfrPopGen', 'id': 50238498, 'node_id': 'MDQ6VXNlcjUwMjM4NDk4', 'avatar_url': 'https://avatars.githubusercontent.com/u/50238498?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/InfrPopGen', 'html_url': 'https://github.com/InfrPopGen', 'followers_url': 'https://api.github.com/users/InfrPopGen/followers', 'following_url': 'https://api.github.com/users/InfrPopGen/following{/other_user}', 'gists_url': 'https://api.github.com/users/InfrPopGen/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/InfrPopGen/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/InfrPop

{'url': 'https://api.github.com/repos/cov-lineages/pango-designation/milestones/502', 'html_url': 'https://github.com/cov-lineages/pango-designation/milestone/502', 'labels_url': 'https://api.github.com/repos/cov-lineages/pango-designation/milestones/502/labels', 'id': 8529818, 'node_id': 'MI_kwDOEGMSVc4Agiea', 'number': 502, 'title': 'CN.1', 'description': None, 'creator': {'login': 'corneliusroemer', 'id': 25161793, 'node_id': 'MDQ6VXNlcjI1MTYxNzkz', 'avatar_url': 'https://avatars.githubusercontent.com/u/25161793?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/corneliusroemer', 'html_url': 'https://github.com/corneliusroemer', 'followers_url': 'https://api.github.com/users/corneliusroemer/followers', 'following_url': 'https://api.github.com/users/corneliusroemer/following{/other_user}', 'gists_url': 'https://api.github.com/users/corneliusroemer/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/corneliusroemer/starred{/owner}{/repo}', 'subscriptions_url': 'h

{'url': 'https://api.github.com/repos/cov-lineages/pango-designation/milestones/602', 'html_url': 'https://github.com/cov-lineages/pango-designation/milestone/602', 'labels_url': 'https://api.github.com/repos/cov-lineages/pango-designation/milestones/602/labels', 'id': 8832707, 'node_id': 'MI_kwDOEGMSVc4AhsbD', 'number': 602, 'title': 'XBL', 'description': None, 'creator': {'login': 'corneliusroemer', 'id': 25161793, 'node_id': 'MDQ6VXNlcjI1MTYxNzkz', 'avatar_url': 'https://avatars.githubusercontent.com/u/25161793?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/corneliusroemer', 'html_url': 'https://github.com/corneliusroemer', 'followers_url': 'https://api.github.com/users/corneliusroemer/followers', 'following_url': 'https://api.github.com/users/corneliusroemer/following{/other_user}', 'gists_url': 'https://api.github.com/users/corneliusroemer/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/corneliusroemer/starred{/owner}{/repo}', 'subscriptions_url': 'ht

{'url': 'https://api.github.com/repos/cov-lineages/pango-designation/milestones/702', 'html_url': 'https://github.com/cov-lineages/pango-designation/milestone/702', 'labels_url': 'https://api.github.com/repos/cov-lineages/pango-designation/milestones/702/labels', 'id': 9201167, 'node_id': 'MI_kwDOEGMSVc4AjGYP', 'number': 702, 'title': 'EK.3', 'description': None, 'creator': {'login': 'corneliusroemer', 'id': 25161793, 'node_id': 'MDQ6VXNlcjI1MTYxNzkz', 'avatar_url': 'https://avatars.githubusercontent.com/u/25161793?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/corneliusroemer', 'html_url': 'https://github.com/corneliusroemer', 'followers_url': 'https://api.github.com/users/corneliusroemer/followers', 'following_url': 'https://api.github.com/users/corneliusroemer/following{/other_user}', 'gists_url': 'https://api.github.com/users/corneliusroemer/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/corneliusroemer/starred{/owner}{/repo}', 'subscriptions_url': 'h

{'url': 'https://api.github.com/repos/cov-lineages/pango-designation/milestones/802', 'html_url': 'https://github.com/cov-lineages/pango-designation/milestone/802', 'labels_url': 'https://api.github.com/repos/cov-lineages/pango-designation/milestones/802/labels', 'id': 9791155, 'node_id': 'MI_kwDOEGMSVc4AlWaz', 'number': 802, 'title': 'HF.1', 'description': None, 'creator': {'login': 'InfrPopGen', 'id': 50238498, 'node_id': 'MDQ6VXNlcjUwMjM4NDk4', 'avatar_url': 'https://avatars.githubusercontent.com/u/50238498?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/InfrPopGen', 'html_url': 'https://github.com/InfrPopGen', 'followers_url': 'https://api.github.com/users/InfrPopGen/followers', 'following_url': 'https://api.github.com/users/InfrPopGen/following{/other_user}', 'gists_url': 'https://api.github.com/users/InfrPopGen/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/InfrPopGen/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/InfrPop